In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pyspark py4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 56 kB/s 
     |████████████████████████████████| 200 kB 68.4 MB/s 
     |████████████████████████████████| 199 kB 87.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=ad54161c1cce6aed333e9d14f1ffcb2bcf2c20d71e13e370e3cbc04fef4e3376
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('ops').getOrCreate()

In [15]:
df = spark.read.csv("/content/drive/MyDrive/SparkWork/SparkDoc/aapl.csv", inferSchema=True, header=True)

In [16]:
df.show()

+-------------------+------+------+------+------+--------+---------+
|               Date|  Open|  High|   Low| Close|  Volume|Adj Close|
+-------------------+------+------+------+------+--------+---------+
|2008-10-14 00:00:00|116.26| 116.4|103.14|104.08|70749800|   104.08|
|2008-10-13 00:00:00|104.55|110.53|101.02|110.26|54967000|   110.26|
|2008-10-10 00:00:00|  85.7| 100.0|  85.0|  96.8|79260700|     96.8|
|2008-10-09 00:00:00| 93.35|  95.8|  86.6| 88.74|57763700|    88.74|
|2008-10-08 00:00:00| 85.91| 96.33| 85.68| 89.79|78847900|    89.79|
|2008-10-07 00:00:00|100.48| 101.5| 88.95| 89.16|67099000|    89.16|
|2008-10-06 00:00:00| 91.96| 98.78| 87.54| 98.14|75264900|    98.14|
|2008-10-03 00:00:00| 104.0| 106.5| 94.65| 97.07|81942800|    97.07|
|2008-10-02 00:00:00|108.01|108.79| 100.0| 100.1|57477300|    100.1|
|2008-10-01 00:00:00|111.92|112.36|107.39|109.12|46303000|   109.12|
|2008-09-30 00:00:00|108.25| 115.0| 106.3|113.66|58095800|   113.66|
|2008-09-29 00:00:00|119.62|119.68

In [17]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [18]:
df.describe().show()

+-------+-----------------+------------------+-----------------+------------------+--------------------+------------------+
|summary|             Open|              High|              Low|             Close|              Volume|         Adj Close|
+-------+-----------------+------------------+-----------------+------------------+--------------------+------------------+
|  count|             6081|              6081|             6081|              6081|                6081|              6081|
|   mean|46.82351093570132|47.681506331195564|45.91359480348622|46.798618648248734|  1.36398640684098E7|23.529794441703633|
| stddev|33.99351710867853|34.578077150719686|33.27310568082934| 33.94723505279488|1.3521074474193025E7| 37.37560090267473|
|    min|            12.88|             13.19|            12.72|             12.94|               88800|              1.65|
|    max|           200.59|            202.96|            197.8|            199.83|           265069000|            199.83|
+-------

In [23]:
df.head(3)[2]

Row(Date=datetime.datetime(2008, 10, 10, 0, 0), Open=85.7, High=100.0, Low=85.0, Close=96.8, Volume=79260700, Adj Close=96.8)

Filter command


*   Using SQL
*   Using dataframe command



**same results in the following two cells.**

In [27]:
df.filter("close < 500").select('open', 'close').show()

+------+------+
|  open| close|
+------+------+
|116.26|104.08|
|104.55|110.26|
|  85.7|  96.8|
| 93.35| 88.74|
| 85.91| 89.79|
|100.48| 89.16|
| 91.96| 98.14|
| 104.0| 97.07|
|108.01| 100.1|
|111.92|109.12|
|108.25|113.66|
|119.62|105.26|
|124.91|128.24|
| 129.8|131.93|
|127.27|128.71|
|131.85|126.84|
|139.94|131.05|
| 142.6|140.91|
|130.57|134.09|
|138.49|127.83|
+------+------+
only showing top 20 rows



In [29]:
df.filter(df['close']<500).select('open', 'close').show()

+------+------+
|  open| close|
+------+------+
|116.26|104.08|
|104.55|110.26|
|  85.7|  96.8|
| 93.35| 88.74|
| 85.91| 89.79|
|100.48| 89.16|
| 91.96| 98.14|
| 104.0| 97.07|
|108.01| 100.1|
|111.92|109.12|
|108.25|113.66|
|119.62|105.26|
|124.91|128.24|
| 129.8|131.93|
|127.27|128.71|
|131.85|126.84|
|139.94|131.05|
| 142.6|140.91|
|130.57|134.09|
|138.49|127.83|
+------+------+
only showing top 20 rows



### ***Q: "Grab all the rows that have a closing price less than 200 and open price greater than 200"***



*   and will not work in spark dataframe operation instead use '&' or => '|' not => "~"
*   In case of multiple condiations, isolate the conditions by enclosing them with a closed bracket().



# ***and***

In [31]:
df.filter((df['close'] < 200) & (df['open'] > 200) ).show()

+-------------------+------+------+------+------+--------+---------+
|               Date|  Open|  High|   Low| Close|  Volume|Adj Close|
+-------------------+------+------+------+------+--------+---------+
|2007-12-28 00:00:00|200.59|201.56|196.88|199.83|24987400|   199.83|
+-------------------+------+------+------+------+--------+---------+



# ***or***

In [33]:
df.filter((df['close'] < 200) | (df['open'] > 200) ).show() 

+-------------------+------+------+------+------+--------+---------+
|               Date|  Open|  High|   Low| Close|  Volume|Adj Close|
+-------------------+------+------+------+------+--------+---------+
|2008-10-14 00:00:00|116.26| 116.4|103.14|104.08|70749800|   104.08|
|2008-10-13 00:00:00|104.55|110.53|101.02|110.26|54967000|   110.26|
|2008-10-10 00:00:00|  85.7| 100.0|  85.0|  96.8|79260700|     96.8|
|2008-10-09 00:00:00| 93.35|  95.8|  86.6| 88.74|57763700|    88.74|
|2008-10-08 00:00:00| 85.91| 96.33| 85.68| 89.79|78847900|    89.79|
|2008-10-07 00:00:00|100.48| 101.5| 88.95| 89.16|67099000|    89.16|
|2008-10-06 00:00:00| 91.96| 98.78| 87.54| 98.14|75264900|    98.14|
|2008-10-03 00:00:00| 104.0| 106.5| 94.65| 97.07|81942800|    97.07|
|2008-10-02 00:00:00|108.01|108.79| 100.0| 100.1|57477300|    100.1|
|2008-10-01 00:00:00|111.92|112.36|107.39|109.12|46303000|   109.12|
|2008-09-30 00:00:00|108.25| 115.0| 106.3|113.66|58095800|   113.66|
|2008-09-29 00:00:00|119.62|119.68

# ***not***

In [34]:
df.filter((df['close'] < 200) &~ (df['open'] > 200) ).show() 

+-------------------+------+------+------+------+--------+---------+
|               Date|  Open|  High|   Low| Close|  Volume|Adj Close|
+-------------------+------+------+------+------+--------+---------+
|2008-10-14 00:00:00|116.26| 116.4|103.14|104.08|70749800|   104.08|
|2008-10-13 00:00:00|104.55|110.53|101.02|110.26|54967000|   110.26|
|2008-10-10 00:00:00|  85.7| 100.0|  85.0|  96.8|79260700|     96.8|
|2008-10-09 00:00:00| 93.35|  95.8|  86.6| 88.74|57763700|    88.74|
|2008-10-08 00:00:00| 85.91| 96.33| 85.68| 89.79|78847900|    89.79|
|2008-10-07 00:00:00|100.48| 101.5| 88.95| 89.16|67099000|    89.16|
|2008-10-06 00:00:00| 91.96| 98.78| 87.54| 98.14|75264900|    98.14|
|2008-10-03 00:00:00| 104.0| 106.5| 94.65| 97.07|81942800|    97.07|
|2008-10-02 00:00:00|108.01|108.79| 100.0| 100.1|57477300|    100.1|
|2008-10-01 00:00:00|111.92|112.36|107.39|109.12|46303000|   109.12|
|2008-09-30 00:00:00|108.25| 115.0| 106.3|113.66|58095800|   113.66|
|2008-09-29 00:00:00|119.62|119.68

***Q: "Pick Date in which the lowest sale was 195"***

In [41]:
df.filter(df['Low'] == 120.68).select(['date', 'volume']).show()

+-------------------+--------+
|               date|  volume|
+-------------------+--------+
|2008-09-18 00:00:00|59819300|
+-------------------+--------+



# ***Collect***


*   Returns row objects inside a list
* show can only be used to view a change, to have that changed data we must use collect instead of show to get that data to play around. 


In [44]:
result = df.filter(df['Low'] == 120.68).collect()

In [45]:
result

[Row(Date=datetime.datetime(2008, 9, 18, 0, 0), Open=130.57, High=135.43, Low=120.68, Close=134.09, Volume=59819300, Adj Close=134.09)]

In [46]:
result[0]

Row(Date=datetime.datetime(2008, 9, 18, 0, 0), Open=130.57, High=135.43, Low=120.68, Close=134.09, Volume=59819300, Adj Close=134.09)

***we can change this result (row) object into a dictionary***

In [47]:
row = result[0]

In [48]:
dictionary_row = row.asDict()

In [49]:
dictionary_row

{'Date': datetime.datetime(2008, 9, 18, 0, 0),
 'Open': 130.57,
 'High': 135.43,
 'Low': 120.68,
 'Close': 134.09,
 'Volume': 59819300,
 'Adj Close': 134.09}

In [50]:
dictionary_row['Date']

datetime.datetime(2008, 9, 18, 0, 0)

# ***GroupBy and Aggregate functions***